In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://localhost:3030/example2/sparql")

#Get 10000 values of variable one
sparql.setQuery("""
PREFIX ex2: <http://www.semanticweb.org/asus/ontologies/2021/4/example2#>
PREFIX srr: <https://w3id.org/srr#>
SELECT ?v1 
where{
        ex2:Variable1  srr:NumericalValue ?v1. 
    }
""")
sparql.setReturnFormat(JSON)
results_v1 = sparql.query().convert()

#Get 10000 values of variable two
sparql.setQuery("""
PREFIX ex2: <http://www.semanticweb.org/asus/ontologies/2021/4/example2#>
PREFIX srr: <https://w3id.org/srr#>
SELECT ?v2 
where{
        ex2:Variable2  srr:NumericalValue ?v2. 
    }
""")
sparql.setReturnFormat(JSON)
results_v2 = sparql.query().convert()

In [3]:
import pandas as pd

#Transform JSON text into CSV files
results_v1_df = pd.io.json.json_normalize(results_v1['results']['bindings'])
results_v1_df['v1.value'].to_csv("C:/Users/asus/v1.csv",index=False)

results_v2_df = pd.io.json.json_normalize(results_v2['results']['bindings'])
results_v2_df['v2.value'].to_csv("C:/Users/asus/v2.csv",index=False)

In [4]:
import matlab.engine
eng = matlab.engine.start_matlab()

#MATLAB reads CSV files
eng.eval("T1 = readtable('v1.csv');",nargout=0)
eng.eval("T2 = readtable('v2.csv');",nargout=0)

eng.eval("S1 = table2struct(T1,'ToScalar',true);",nargout=0)
eng.eval("S2 = table2struct(T2,'ToScalar',true);",nargout=0)

In [14]:
D1 = eng.workspace["S1"]
D2 = eng.workspace["S2"]
v1 = D1['Var1']
v2 = D2['Var1']
xx = [v1,v2]
print(type(xx))

<class 'list'>


In [15]:
xx = matlab.double(xx)
print(type(xx),xx.size)

#Call MCS functions
Pf1 = eng.MCS(xx)
print(Pf1)

<class 'mlarray.double'> (2, 10000)
0.0035


In [19]:
beta1 = eng.norminv(Pf1)
print(-beta1)

2.6968442608781262
